# Human-in-the-Loop

Human-in-the-Loop (HITL) 미들웨어를 사용하면 에이전트 도구 호출에 사람의 감독을 추가할 수 있습니다.

모델이 검토가 필요한 작업(예: 파일 작성 또는 SQL 실행)을 제안하면 미들웨어가 실행을 일시 중지하고 결정을 기다립니다.

## 작동 원리

1. 모델이 도구 호출을 생성
2. 미들웨어가 도구 호출을 검사
3. 사람의 승인이 필요한 경우 **interrupt** 발생
4. 그래프 상태가 저장되고 실행 일시 중지
5. 사람의 결정을 받아 실행 재개

## Interrupt 결정 타입

미들웨어는 세 가지 내장 응답 방식을 정의합니다:

| 결정 타입 | 설명 | 사용 사례 |
|---------|------|----------|
| ✅ `approve` | 작업을 그대로 승인하고 변경 없이 실행 | 작성된 이메일을 정확히 그대로 전송 |
| ✏️ `edit` | 도구 호출을 수정하여 실행 | 이메일 수신자를 변경한 후 전송 |
| ❌ `reject` | 도구 호출을 거부하고 설명 추가 | 이메일 초안을 거부하고 재작성 방법 설명 |

## 사전 준비

환경 변수를 설정합니다.

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

## 기본 예제

민감한 작업에 대해 사람의 승인이 필요한 간단한 에이전트를 만들어봅시다.

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import ChatOpenAI
from langchain.tools import tool

# 도구 정의
@tool
def write_file(filename: str, content: str) -> str:
    """Write content to a file."""
    with open(filename, 'w') as f:
        f.write(content)
    return f"File {filename} written successfully"

@tool
def read_file(filename: str) -> str:
    """Read content from a file."""
    try:
        with open(filename, 'r') as f:
            return f.read()
    except FileNotFoundError:
        return f"File {filename} not found"

@tool
def delete_file(filename: str) -> str:
    """Delete a file."""
    import os
    try:
        os.remove(filename)
        return f"File {filename} deleted successfully"
    except FileNotFoundError:
        return f"File {filename} not found"

model = ChatOpenAI(model="gpt-4.1-mini")

# HITL 미들웨어와 함께 에이전트 생성
agent = create_agent(
    model=model,
    tools=[write_file, read_file, delete_file],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "write_file": True,    # 모든 결정(approve, edit, reject) 허용
                "delete_file": True,   # 모든 결정 허용
                "read_file": False,    # 안전한 작업, 승인 불필요
            },
            description_prefix="Tool execution pending approval",
        ),
    ],
    checkpointer=InMemorySaver(),  # 체크포인터 필수
)

print("Agent created with HITL middleware")

### Interrupt 발생 및 처리

In [ ]:
from langgraph.types import Command

# thread_id를 포함한 config 필수
config = {"configurable": {"thread_id": "thread_001"}}

# 파일 쓰기 요청 (interrupt 발생 예상)
result = agent.invoke(
    {
        "messages": [{
            "role": "user",
            "content": "Write 'Hello World' to a file called test.txt"
        }]
    },
    config=config
)

# Interrupt 확인
if "__interrupt__" in result:
    print("\n=== Interrupt Detected ===")
    interrupt_data = result["__interrupt__"][0].value
    
    print("\nAction Requests:")
    for action in interrupt_data["action_requests"]:
        print(f"  Tool: {action['name']}")
        print(f"  Args: {action['arguments']}")
        print(f"  Description: {action['description']}")
    
    print("\nReview Configs:")
    for config_item in interrupt_data["review_configs"]:
        print(f"  Tool: {config_item['action_name']}")
        print(f"  Allowed decisions: {config_item['allowed_decisions']}")
else:
    print("No interrupt occurred")

## 결정 타입

### 1. Approve (승인)

In [ ]:
# 작업 승인
result = agent.invoke(
    Command(
        resume={"decisions": [{"type": "approve"}]}
    ),
    config=config  # 동일한 thread_id
)

print("Result after approval:")
print(result["messages"][-1].content)

# 파일이 실제로 생성되었는지 확인
import os
if os.path.exists('test.txt'):
    with open('test.txt', 'r') as f:
        print(f"\nFile content: {f.read()}")

### 2. Edit (수정)

In [ ]:
# 새로운 파일 쓰기 요청
config2 = {"configurable": {"thread_id": "thread_002"}}

result = agent.invoke(
    {
        "messages": [{
            "role": "user",
            "content": "Write 'Original content' to original.txt"
        }]
    },
    config=config2
)

if "__interrupt__" in result:
    print("Interrupt detected - modifying the action")
    
    # 인수를 수정하여 승인
    result = agent.invoke(
        Command(
            resume={
                "decisions": [{
                    "type": "edit",
                    "edited_action": {
                        "name": "write_file",
                        "args": {
                            "filename": "modified.txt",  # 파일명 변경
                            "content": "Modified content"  # 내용 변경
                        }
                    }
                }]
            }
        ),
        config=config2
    )
    
    print("\nResult after edit:")
    print(result["messages"][-1].content)
    
    # 수정된 파일 확인
    if os.path.exists('modified.txt'):
        with open('modified.txt', 'r') as f:
            print(f"\nModified file content: {f.read()}")

### 3. Reject (거부)

In [ ]:
# 파일 삭제 요청
config3 = {"configurable": {"thread_id": "thread_003"}}

result = agent.invoke(
    {
        "messages": [{
            "role": "user",
            "content": "Delete the file test.txt"
        }]
    },
    config=config3
)

if "__interrupt__" in result:
    print("Interrupt detected - rejecting the action")
    
    # 작업 거부 및 피드백 제공
    result = agent.invoke(
        Command(
            resume={
                "decisions": [{
                    "type": "reject",
                    "message": "I cannot delete this file because it contains important data. Please back it up first."
                }]
            }
        ),
        config=config3
    )
    
    print("\nResult after rejection:")
    print(result["messages"][-1].content)

## 실용적인 예제: 데이터베이스 관리 에이전트

데이터베이스 작업에 대한 승인이 필요한 에이전트를 만들어봅시다.

In [ ]:
@tool
def execute_sql(query: str) -> str:
    """Execute SQL query on the database."""
    # 실제로는 데이터베이스에 연결하여 실행
    print(f"Executing SQL: {query}")
    return f"Query executed: {query}"

@tool
def read_table(table_name: str) -> str:
    """Read data from a table."""
    return f"Reading data from {table_name}"

@tool
def backup_database() -> str:
    """Backup the entire database."""
    return "Database backup completed"

# 데이터베이스 에이전트 생성
db_agent = create_agent(
    model=model,
    tools=[execute_sql, read_table, backup_database],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                # SQL 실행은 승인 또는 거부만 가능 (편집 불가)
                "execute_sql": {"allowed_decisions": ["approve", "reject"]},
                # 백업은 모든 결정 허용
                "backup_database": True,
                # 읽기는 승인 불필요
                "read_table": False,
            },
            description_prefix="Database operation pending approval",
        ),
    ],
    checkpointer=InMemorySaver(),
)

print("Database agent created")

### 안전한 작업 (Interrupt 없음)

In [ ]:
config_db1 = {"configurable": {"thread_id": "db_thread_001"}}

# 읽기 작업 - interrupt 발생하지 않음
result = db_agent.invoke(
    {
        "messages": [{
            "role": "user",
            "content": "Read data from users table"
        }]
    },
    config=config_db1
)

print("Result (no interrupt):")
print(result["messages"][-1].content)
print(f"\nInterrupt occurred: {'__interrupt__' in result}")

### 위험한 작업 (Interrupt 발생)

In [ ]:
config_db2 = {"configurable": {"thread_id": "db_thread_002"}}

# SQL 실행 - interrupt 발생
result = db_agent.invoke(
    {
        "messages": [{
            "role": "user",
            "content": "Delete all records older than 30 days from the logs table"
        }]
    },
    config=config_db2
)

if "__interrupt__" in result:
    print("=== Interrupt for SQL Execution ===")
    interrupt_data = result["__interrupt__"][0].value
    
    action = interrupt_data["action_requests"][0]
    print(f"\nSQL Query: {action['arguments']['query']}")
    print(f"\nAllowed decisions: {interrupt_data['review_configs'][0]['allowed_decisions']}")
    
    # 위험한 작업 거부
    result = db_agent.invoke(
        Command(
            resume={
                "decisions": [{
                    "type": "reject",
                    "message": "This DELETE operation is too broad. Please add a LIMIT clause or be more specific about which records to delete."
                }]
            }
        ),
        config=config_db2
    )
    
    print("\nResult after rejection:")
    print(result["messages"][-1].content)

## 다중 작업 승인

여러 도구 호출이 동시에 일시 중지되면 각 작업에 대해 별도의 결정을 제공해야 합니다.

In [ ]:
@tool
def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email."""
    return f"Email sent to {recipient} with subject '{subject}'"

@tool
def schedule_meeting(participants: list, time: str) -> str:
    """Schedule a meeting."""
    return f"Meeting scheduled at {time} with {', '.join(participants)}"

@tool
def create_document(title: str, content: str) -> str:
    """Create a new document."""
    return f"Document '{title}' created"

# 모든 작업에 승인이 필요한 에이전트
multi_agent = create_agent(
    model=model,
    tools=[send_email, schedule_meeting, create_document],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email": True,
                "schedule_meeting": True,
                "create_document": True,
            },
        ),
    ],
    checkpointer=InMemorySaver(),
)

config_multi = {"configurable": {"thread_id": "multi_thread_001"}}

# 여러 작업을 동시에 요청
result = multi_agent.invoke(
    {
        "messages": [{
            "role": "user",
            "content": "Send an email to john@example.com about the project update, then schedule a meeting with the team for tomorrow at 2pm"
        }]
    },
    config=config_multi
)

if "__interrupt__" in result:
    print("=== Multiple Actions Require Approval ===")
    interrupt_data = result["__interrupt__"][0].value
    
    print(f"\nNumber of actions: {len(interrupt_data['action_requests'])}")
    for i, action in enumerate(interrupt_data["action_requests"]):
        print(f"\nAction {i+1}:")
        print(f"  Tool: {action['name']}")
        print(f"  Args: {action['arguments']}")
    
    # 각 작업에 대한 결정 제공
    result = multi_agent.invoke(
        Command(
            resume={
                "decisions": [
                    {"type": "approve"},  # 첫 번째 작업 승인
                    {
                        "type": "edit",  # 두 번째 작업 수정
                        "edited_action": {
                            "name": "schedule_meeting",
                            "args": {
                                "participants": ["john@example.com", "jane@example.com"],
                                "time": "tomorrow at 3pm"  # 시간 변경
                            }
                        }
                    }
                ]
            }
        ),
        config=config_multi
    )
    
    print("\n=== Result After Decisions ===")
    print(result["messages"][-1].content)

## 실행 라이프사이클

미들웨어는 모델이 응답을 생성한 후, 도구 호출이 실행되기 전에 실행되는 `after_model` 훅을 정의합니다:

1. 에이전트가 모델을 호출하여 응답 생성
2. 미들웨어가 응답에서 도구 호출 검사
3. 사람의 입력이 필요한 호출이 있으면 `HITLRequest`를 빌드하고 `interrupt` 호출
4. 에이전트가 사람의 결정을 기다림
5. `HITLResponse` 결정에 따라 미들웨어가 승인/편집된 호출을 실행하거나 거부된 호출에 대한 `ToolMessage` 합성 후 실행 재개

## 고급 예제: 조건부 승인 정책

상황에 따라 다른 승인 정책을 적용할 수 있습니다.

In [ ]:
@tool
def transfer_money(amount: float, recipient: str) -> str:
    """Transfer money to a recipient."""
    return f"Transferred ${amount} to {recipient}"

@tool
def check_balance(account: str) -> str:
    """Check account balance."""
    return f"Account {account} balance: $10,000"

# 금융 에이전트
finance_agent = create_agent(
    model=model,
    tools=[transfer_money, check_balance],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                # 금전 이체는 승인 또는 거부만 (편집 불가)
                "transfer_money": {"allowed_decisions": ["approve", "reject"]},
                # 잔액 조회는 승인 불필요
                "check_balance": False,
            },
            description_prefix="Financial operation pending approval",
        ),
    ],
    checkpointer=InMemorySaver(),
)

config_finance = {"configurable": {"thread_id": "finance_thread_001"}}

# 금전 이체 요청
result = finance_agent.invoke(
    {
        "messages": [{
            "role": "user",
            "content": "Transfer $500 to alice@example.com"
        }]
    },
    config=config_finance
)

if "__interrupt__" in result:
    print("=== Financial Operation Requires Approval ===")
    interrupt_data = result["__interrupt__"][0].value
    action = interrupt_data["action_requests"][0]
    
    amount = action["arguments"]["amount"]
    recipient = action["arguments"]["recipient"]
    
    print(f"\nAmount: ${amount}")
    print(f"Recipient: {recipient}")
    
    # 금액에 따른 조건부 승인
    if amount > 1000:
        decision = {
            "type": "reject",
            "message": f"Transfer amount ${amount} exceeds the $1,000 limit. Please contact a supervisor."
        }
        print("\n❌ REJECTED: Amount exceeds limit")
    else:
        decision = {"type": "approve"}
        print("\n✅ APPROVED: Amount within limit")
    
    result = finance_agent.invoke(
        Command(resume={"decisions": [decision]}),
        config=config_finance
    )
    
    print("\nFinal result:")
    print(result["messages"][-1].content)

## 정리 및 모범 사례

### 1. 체크포인터 필수

HITL을 사용하려면 체크포인터를 설정해야 합니다. 프로덕션에서는 `AsyncPostgresSaver`를 사용하세요.

In [ ]:
# 개발/테스트
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()

# 프로덕션 (예제)
# from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
# checkpointer = AsyncPostgresSaver.from_conn_string("postgresql://...")

### 2. Thread ID 관리

동일한 thread_id로 대화를 일시 중지하고 재개해야 합니다.

In [ ]:
# 동일한 thread_id 사용
config = {"configurable": {"thread_id": "unique_thread_id"}}

# 초기 호출
result = agent.invoke({"messages": [...]}, config=config)

# 재개
result = agent.invoke(Command(resume={...}), config=config)  # 동일한 config

### 3. 결정 순서

여러 작업에 대한 결정은 interrupt에 나타난 순서와 동일해야 합니다.

In [ ]:
# 올바른 순서
decisions = [
    {"type": "approve"},     # 첫 번째 작업
    {"type": "reject", ...}, # 두 번째 작업
    {"type": "edit", ...}    # 세 번째 작업
]

### 4. 편집 시 주의사항

도구 인수를 편집할 때는 신중하게 변경하세요. 큰 수정은 모델이 접근 방식을 재평가하고 예기치 않은 동작을 일으킬 수 있습니다.

In [ ]:
# 좋은 예: 소폭 수정
{
    "type": "edit",
    "edited_action": {
        "name": "send_email",
        "args": {
            "recipient": "new@example.com",  # 수신자만 변경
            "subject": "...",  # 나머지는 유지
            "body": "..."
        }
    }
}

# 나쁜 예: 대폭 수정
# 도구 이름 변경, 모든 인수 변경 등

### 5. 적절한 승인 정책

작업의 위험도에 따라 적절한 승인 정책을 설정하세요.

In [ ]:
HumanInTheLoopMiddleware(
    interrupt_on={
        # 위험한 작업: 승인/거부만 (편집 불가)
        "delete_database": {"allowed_decisions": ["approve", "reject"]},
        
        # 보통 위험: 모든 결정 허용
        "send_email": True,
        
        # 안전한 작업: 승인 불필요
        "read_data": False,
    }
)

## 요약

Human-in-the-Loop을 사용하면:

1. **민감한 작업에 대한 사람의 감독**을 추가할 수 있습니다
2. **Approve, Edit, Reject** 세 가지 결정 타입을 지원합니다
3. **체크포인터를 통한 상태 저장**으로 안전한 일시 중지/재개가 가능합니다
4. **작업별 승인 정책**을 유연하게 설정할 수 있습니다

HITL은 AI 에이전트가 안전하고 책임감 있게 작동하도록 보장하는 중요한 메커니즘입니다.